In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup

In [ ]:
ndim = 15

In [ ]:
import time

import emcee
import numpy as np

import pymc3 as pm
from pymc3.step_methods.hmc import quadpotential as quad

In [ ]:
np.random.seed(41)

with pm.Model() as model:
    pm.Normal("x", shape=ndim)

    potential = quad.QuadPotentialDiag(np.ones(ndim))

    step_kwargs = dict()
    step_kwargs["model"] = model
    step_kwargs["step_scale"] = 1.0 * model.ndim ** 0.25
    step_kwargs["adapt_step_size"] = False
    step = pm.NUTS(potential=potential, **step_kwargs)

    start = time.time()
    trace = pm.sample(tune=0, draws=10000, step=step, cores=1)
    time_pymc3 = time.time() - start

In [ ]:
samples_pymc3 = np.array(trace.get_values("x", combine=False))
samples_pymc3 = np.moveaxis(samples_pymc3, 0, 1)
tau_pymc3 = emcee.autocorr.integrated_time(samples_pymc3)
neff_pymc3 = np.prod(samples_pymc3.shape[:2]) / tau_pymc3
teff_pymc3 = time_pymc3 / neff_pymc3
teff_pymc3

In [ ]:
np.random.seed(1234)

import exoplanet as xo

with model:
    func = xo.get_theano_function_for_var(model.logpt)

    def logprob(theta):
        point = model.bijection.rmap(theta)
        args = xo.get_args_for_theano_function(point)
        return func(*args)

    x0 = np.random.randn(ndim)
    nwalkers = 36
    x0 = np.random.randn(nwalkers, ndim)

    emcee_sampler = emcee.EnsembleSampler(nwalkers, ndim, logprob)
    state = emcee_sampler.run_mcmc(x0, 2000, progress=True)
    emcee_sampler.reset()
    strt = time.time()
    emcee_sampler.run_mcmc(state, 20000, progress=True)
    time_emcee = time.time() - strt

In [ ]:
samples_emcee = emcee_sampler.get_chain()
tau_emcee = emcee.autocorr.integrated_time(samples_emcee)
neff_emcee = np.prod(samples_emcee.shape[:2]) / tau_emcee
teff_emcee = time_emcee / neff_emcee
teff_emcee

In [ ]:
def get_function(x):
    n_t, n_w = x.shape
    f = np.zeros(n_t)
    for k in range(n_w):
        f += emcee.autocorr.function_1d(x[:, k])
    f /= n_w
    return f


fig, axes = plt.subplots(2, 2, figsize=(10, 8))

ax = axes[0, 0]
t_emcee = np.linspace(0, time_emcee, samples_emcee.shape[0])
m = t_emcee < 5
ax.plot(t_emcee[m], samples_emcee[m, 5, 0], lw=0.75)
ax.annotate(
    "emcee",
    xy=(0, 1),
    xycoords="axes fraction",
    ha="left",
    va="top",
    fontsize=16,
    xytext=(5, -5),
    textcoords="offset points",
)
ax.set_ylabel(r"$\theta_1$")
ax.set_xlim(0, 5)
ax.set_ylim(-4, 4)
ax.set_xticklabels([])

ax = axes[1, 0]
t_pymc3 = np.linspace(0, time_pymc3, samples_pymc3.shape[0])
m = t_pymc3 < 5
ax.plot(t_pymc3[m], samples_pymc3[m, 0, 0], lw=0.75)
ax.annotate(
    "pymc3",
    xy=(0, 1),
    xycoords="axes fraction",
    ha="left",
    va="top",
    fontsize=16,
    xytext=(5, -5),
    textcoords="offset points",
)
ax.set_ylabel(r"$\theta_1$")
ax.set_xlabel("walltime [s]")
ax.set_xlim(0, 5)
ax.set_ylim(-4, 4)

ax = axes[0, 1]
f_emcee = get_function(samples_emcee[:, :, 0])
scale = 1e3 * time_emcee / np.prod(samples_emcee.shape[:2])
ax.plot(scale * np.arange(len(f_emcee)), f_emcee)
ax.axhline(0, color="k", lw=0.5)
ax.annotate(
    "emcee",
    xy=(1, 1),
    xycoords="axes fraction",
    ha="right",
    va="top",
    fontsize=16,
    xytext=(-5, -5),
    textcoords="offset points",
)
val = 2 * scale * tau_emcee[0]
max_x = 1.5 * val
ax.axvline(val, color="k", lw=3, alpha=0.3)
ax.annotate(
    "{0:.1f} ms".format(val),
    xy=(val / max_x, 1),
    xycoords="axes fraction",
    ha="right",
    va="top",
    fontsize=12,
    alpha=0.3,
    xytext=(-5, -5),
    textcoords="offset points",
)
ax.set_xlim(0, max_x)
ax.set_xticklabels([])
ax.set_yticks([])
ax.set_ylabel("autocorrelation")

ax = axes[1, 1]
f_pymc3 = get_function(samples_pymc3[:, :, 0])
scale = 1e3 * time_pymc3 / np.prod(samples_pymc3.shape[:2])
ax.plot(scale * np.arange(len(f_pymc3)), f_pymc3)
ax.axhline(0, color="k", lw=0.5)
ax.annotate(
    "pymc3",
    xy=(1, 1),
    xycoords="axes fraction",
    ha="right",
    va="top",
    fontsize=16,
    xytext=(-5, -5),
    textcoords="offset points",
)
val = 2 * scale * tau_pymc3[0]
ax.axvline(val, color="k", lw=3, alpha=0.3)
ax.annotate(
    "{0:.1f} ms".format(val),
    xy=(val / max_x, 1),
    xycoords="axes fraction",
    ha="left",
    va="top",
    fontsize=12,
    alpha=0.3,
    xytext=(5, -5),
    textcoords="offset points",
)
ax.set_xlim(0, max_x)
ax.set_yticks([])
ax.set_xlabel("walltime lag [ms]")
ax.set_ylabel("autocorrelation")

fig.subplots_adjust(hspace=0.05, wspace=0.15)
fig.savefig("gaussians.pdf", bbox_inches="tight");